In [3]:
import pandas as pd
import sqlite3
import json

# --- STEP 1: LOAD CSV DATA ---
print("Loading orders.csv...")
orders_df = pd.read_csv("orders.csv")

# --- STEP 2: LOAD JSON DATA ---
print("Loading users.json...")
users_df = pd.read_json("users.json")

# --- STEP 3: LOAD SQL DATA ---
print("Processing restaurants.sql...")
# We create a blank database in RAM to 'run' the .sql script
conn = sqlite3.connect(":memory:")

# Read the .sql text file and execute the commands inside it
with open("restaurants.sql","r") as f:
    sql_script = f.read()
conn.executescript(sql_script)

# Pull the newly created table into a Pandas DataFrame
restaurants_df = pd.read_sql_query("SELECT * FROM restaurants", conn)

# --- STEP 4: MERGE THE DATA (Joins) ---
print("Merging datasets...")

# Merge 1: orders.user_id -> users.user_id
# We use 'left' join to keep all orders even if user info is missing
merged_intermediate = pd.merge(orders_df, users_df, on="user_id", how="left")

# Merge 2: merged_intermediate.restaurant_id -> restaurants.restaurant_id
final_df = pd.merge(merged_intermediate, restaurants_df, on="restaurant_id", how="left")

# --- STEP 5: CREATE FINAL DATASET ---
output_filename = "final_food_delivery_dataset.csv"
final_df.to_csv(output_filename, index=False)

print("-" * 30)
print(f"SUCCESS! Created: {output_filename}")
print("-" * 30)

# --- STEP 6: QUICK PREVIEW (For your analysis) ---
print("Preview of the final merged data:")
print(final_df.head())

# Close the SQL connection
conn.close()

# 1. Order trends over time
# Ensure the date column is in the correct format
final_df["order_date"] = pd.to_datetime(final_df["order_date"])
daily_orders = final_df.groupby("order_date").size()

print("--- Order Trends (Total orders per day) ---")
print(daily_orders.head())

# 2. User behavior (Top 5 customers)
top_users = final_df.groupby("user_id").size().sort_values(ascending=False)
print("\n--- User Behavior (Most frequent customers) ---")
print(top_users.head(5))


Loading orders.csv...
Loading users.json...
Processing restaurants.sql...
Merging datasets...
------------------------------
SUCCESS! Created: final_food_delivery_dataset.csv
------------------------------
Preview of the final merged data:
   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                restaurant_name_x       name       city membership  \
0               New Foods Chinese  User_2508  Hyderabad    Regular   
1  Ruchi Curry House Multicuisine  User_2693       Pune    Regular   
2           Spice Kitchen Punjabi  User_2084    Chennai       Gold   
3          Darbar Kitchen Non-Veg   User_319  Bangalore       Gold   
4       Royal Eater

/tmp/ipython-input-3407537494.py:53: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df["order_date"] = pd.to_datetime(final_df["order_date"])
